In [1]:
pip install -U bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import re
from collections import Counter
import random
from transformers import pipeline, set_seed
from torch.nn import functional as F
import math

In [3]:
from google.colab import files
import os
required_files = ["prefiksy.txt"]
for f in required_files:
    if not os.path.exists(f):
        files.upload()

Saving prefiksy.txt to prefiksy.txt


In [4]:
with open("prefiksy.txt", "r") as f:
    prefixes_str = f.read()

In [5]:
prefixes = prefixes_str.splitlines()
prefixes[:5]

['Obowiązuje on od',
 'Został zrodzony ze',
 'Po pierwsze, projekt',
 'Po Panthers przejechali',
 'Duze dwusuwowe diesle']

In [6]:
model_name = "eryk-mazus/polka-1.1b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
p = random.choice(prefixes)
p.split(" ")[0]

'Pan'

In [43]:
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

def normalize_sentence_prob(sentence):
    return int(1000 * sentence_prob(sentence).item() / math.log(len(sentence)))


In [15]:
s1 = "Pan pali"
s2 = "Pan pali papierosy"
s3 = "Pan pali papierosy."

print(sentence_prob(s1) / math.log(len(s1)))
print(sentence_prob(s2) / math.log(len(s2)))
print(sentence_prob(s3) / math.log(len(s3)))


-8.220356118398573
-6.978989794268764
-7.997066730405136


In [35]:
random.choice(prefixes)

'Mam mocno mieszane'

In [67]:
def generate_one_letter_sentences():
  sentences_ready = []
  sentences_in_progress = [(random.choice(prefixes), -100000)]
  temperature_values = [0.2, 0.5]
  top_p_values = [0.7, 0.9]
  top_k_values = [5, 10]
  threshold = -13000
  # temperature_values = [0.2]
  # top_p_values = [0.7]
  # top_k_values = [5]
  for temp in temperature_values:
    for top_p in top_p_values:
      for top_k in top_k_values:
        for p, prob in sentences_in_progress:
          try:
            sentences_in_progress.remove((p, prob))
          except:
            print(f"ERROR: {sentences_in_progress}, {p}, {prob}")
            return sorted(sentences_ready, key=lambda x: x[1], reverse=True)
          p_last_word = p.split(" ")[-1]
          first_letter_lower = p.split(" ")[0][0].lower()
          prompt_lower = f"{p} {first_letter_lower}"
          prompt_upper = f"{p} {first_letter_lower.upper()}"
          prompts = [prompt_lower, prompt_upper]

          for prompt in prompts:
            model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
            with torch.no_grad():
              generated_ids = model.generate(
                **model_inputs,
                max_new_tokens= int(len(prompt)),
                do_sample=True,
                penalty_alpha=0.6,
                top_k=top_k,
                temperature=temp,
                top_p=top_p,
              )
            output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            output = output.replace("\\n", " ")
            previous_words = output[:len(p)]
            generated_words = output[len(p):].split(" ")
            while len(generated_words) > 0 and generated_words[0].lower() in p.lower():
              generated_words = generated_words[1:]
            # print(f"output: {output}")
            # print(f"previous words: {previous_words}")
            # print(f"generated words: {generated_words}")
            print()
            print(f"READY: {sentences_ready}")
            for w in generated_words:
              if w == "":
                continue
              if w[0] == first_letter_lower or w[0] == first_letter_lower.upper():
                previous_words = f"{previous_words} {w}"
                if previous_words[-1] == "." or previous_words[-1] == "?" or previous_words[-1] == "!":
                  print("NATURALNY DOT")
                  sentences_ready.append((previous_words, normalize_sentence_prob(previous_words)))
                elif len(previous_words.split(" ")) <= 8:
                  sentences_in_progress.append((previous_words, normalize_sentence_prob(previous_words)))
                  # previous_words += f"{w} "
                if normalize_sentence_prob(previous_words + ".") > threshold:
                  sentences_ready.append((previous_words + ".", 10 * normalize_sentence_prob(previous_words + ".")))
                  print("WYMUSZONY DOT")
              else:
                break
            print(sorted(sentences_in_progress, key=lambda x: x[1], reverse=True))
            sentences_in_progress = sorted(sentences_in_progress, key=lambda x: x[1], reverse=True)[:7]
            print(f"sent in prog: {sentences_in_progress}")


  return sorted(sentences_ready, key=lambda x: x[1], reverse=True)


In [69]:
out1 = generate_one_letter_sentences()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: []
WYMUSZONY DOT


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


WYMUSZONY DOT
[('Poprawki proponowane przez pana', -7862), ('Poprawki proponowane przez pana prezydenta', -7913)]
sent in prog: [('Poprawki proponowane przez pana', -7862), ('Poprawki proponowane przez pana prezydenta', -7913)]

READY: [('Poprawki proponowane przez pana.', -101800), ('Poprawki proponowane przez pana prezydenta.', -92800)]
WYMUSZONY DOT


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


READY: [('Poprawki proponowane przez pana.', -101800), ('Poprawki proponowane przez pana prezydenta.', -92800), ('Poprawki proponowane przez Pana.', -99000), ('Poprawki proponowane przez Pana Prezydenta.', -92390), ('Poprawki proponowane przez pana prezydenta Pawła.', -112730), ('Poprawki proponowane przez Pana Pawła.', -106350), ('Poprawki proponowane przez pana Pawła.', -110560), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego.', -114330), ('Poprawki proponowane przez Pana Prezydenta podyktowane.', -125000), ('Poprawki proponowane przez Pana Prezydenta Pawła.', -111450), ('Poprawki proponowane przez pana Pawła Piskorskiego.', -110070), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego pn.', -12621), ('Poprawki proponowane przez Pana Pawła Piskorskiego.', -107130)]
[('Poprawki proponowane przez Pana Prezydenta Pawła', -9008), ('Poprawki proponowane przez Pana Pawła Piskorskiego', -9561), ('Poprawki proponowane przez pana Pawła Piskorskiego padające', -11

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Poprawki proponowane przez pana.', -101800), ('Poprawki proponowane przez pana prezydenta.', -92800), ('Poprawki proponowane przez Pana.', -99000), ('Poprawki proponowane przez Pana Prezydenta.', -92390), ('Poprawki proponowane przez pana prezydenta Pawła.', -112730), ('Poprawki proponowane przez Pana Pawła.', -106350), ('Poprawki proponowane przez pana Pawła.', -110560), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego.', -114330), ('Poprawki proponowane przez Pana Prezydenta podyktowane.', -125000), ('Poprawki proponowane przez Pana Prezydenta Pawła.', -111450), ('Poprawki proponowane przez pana Pawła Piskorskiego.', -110070), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego pn.', -12621), ('Poprawki proponowane przez Pana Pawła Piskorskiego.', -107130)]
[('Poprawki proponowane przez Pana Prezydenta Pawła', -9008), ('Poprawki proponowane przez Pana Pawła Piskorskiego', -9561), ('Poprawki proponowane przez pana Pawła Piskorskiego padające', -11

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Poprawki proponowane przez pana.', -101800), ('Poprawki proponowane przez pana prezydenta.', -92800), ('Poprawki proponowane przez Pana.', -99000), ('Poprawki proponowane przez Pana Prezydenta.', -92390), ('Poprawki proponowane przez pana prezydenta Pawła.', -112730), ('Poprawki proponowane przez Pana Pawła.', -106350), ('Poprawki proponowane przez pana Pawła.', -110560), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego.', -114330), ('Poprawki proponowane przez Pana Prezydenta podyktowane.', -125000), ('Poprawki proponowane przez Pana Prezydenta Pawła.', -111450), ('Poprawki proponowane przez pana Pawła Piskorskiego.', -110070), ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego pn.', -12621), ('Poprawki proponowane przez Pana Pawła Piskorskiego.', -107130)]
[('Poprawki proponowane przez Pana Prezydenta Pawła', -9008), ('Poprawki proponowane przez Pana Pawła Piskorskiego', -9561), ('Poprawki proponowane przez pana Pawła Piskorskiego padające', -11

In [70]:
out1

[('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego pn.', -12621),
 ('Poprawki proponowane przez Pana Prezydenta.', -92390),
 ('Poprawki proponowane przez pana prezydenta.', -92800),
 ('Poprawki proponowane przez Pana.', -99000),
 ('Poprawki proponowane przez pana.', -101800),
 ('Poprawki proponowane przez Pana Pawła.', -106350),
 ('Poprawki proponowane przez Pana Pawła Piskorskiego.', -107130),
 ('Poprawki proponowane przez pana Pawła Piskorskiego.', -110070),
 ('Poprawki proponowane przez pana Pawła.', -110560),
 ('Poprawki proponowane przez Pana Prezydenta Pawła.', -111450),
 ('Poprawki proponowane przez pana prezydenta Pawła.', -112730),
 ('Poprawki proponowane przez pana prezydenta Pawła Piskorskiego.', -114330),
 ('Poprawki proponowane przez Pana Prezydenta podyktowane.', -125000)]

In [73]:
out2 = generate_one_letter_sentences()


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: []
[]
sent in prog: []

READY: []


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


WYMUSZONY DOT
[('Przywołuję panią poseł Pawłowicz', -10675)]
sent in prog: [('Przywołuję panią poseł Pawłowicz', -10675)]

READY: [('Przywołuję panią poseł Pawłowicz.', -114390)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


READY: [('Przywołuję panią poseł Pawłowicz.', -114390)]
[('Przywołuję panią poseł Pawłowicz posłankę Prawa pracy Pawłowicz,', -17056)]
sent in prog: [('Przywołuję panią poseł Pawłowicz posłankę Prawa pracy Pawłowicz,', -17056)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Przywołuję panią poseł Pawłowicz.', -114390)]
[('Przywołuję panią poseł Pawłowicz posłankę Prawa pracy Pawłowicz,', -17056)]
sent in prog: [('Przywołuję panią poseł Pawłowicz posłankę Prawa pracy Pawłowicz,', -17056)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Przywołuję panią poseł Pawłowicz.', -114390)]
[]
sent in prog: []

READY: [('Przywołuję panią poseł Pawłowicz.', -114390)]
[]
sent in prog: []


In [76]:
out3 = generate_one_letter_sentences()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: []


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


WYMUSZONY DOT
[('Obejmował on oprócz oświaty', -10448)]
sent in prog: [('Obejmował on oprócz oświaty', -10448)]

READY: [('Obejmował on oprócz oświaty.', -127070)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


READY: [('Obejmował on oprócz oświaty.', -127070)]
[('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz Oddziału oświatowego, Oddziału Oświaty Obywatelskiej,', -15339), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]
sent in prog: [('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz Oddziału oświatowego, Oddziału Oświaty Obywatelskiej,', -15339), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Obejmował on oprócz oświaty.', -127070)]
[('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz Oddziału oświatowego, Oddziału Oświaty Obywatelskiej,', -15339), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]
sent in prog: [('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz Oddziału oświatowego, Oddziału Oświaty Obywatelskiej,', -15339), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Obejmował on oprócz oświaty.', -127070)]
[('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]
sent in prog: [('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]

READY: [('Obejmował on oprócz oświaty.', -127070)]
[('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oświaty Oraz oświatę, opieki', -15248), ('Obejmował on oprócz oświaty Oraz Ochrony', -16010)]
sent in prog: [('Obejmował on oprócz Oddziału Obrony Ojczyzny, Oddziału', -13975), ('Obejmował on oprócz oświaty Oraz oświatę,', -15071), ('Obejmował on oprócz oś

In [77]:
out3

[('Obejmował on oprócz oświaty.', -127070)]

In [78]:
out4 = generate_one_letter_sentences()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: []


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


WYMUSZONY DOT
[('Doskonały dostęp do dachu', -8494)]
sent in prog: [('Doskonały dostęp do dachu', -8494)]

READY: [('Doskonały dostęp do dachu.', -94900)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


READY: [('Doskonały dostęp do dachu.', -94900), ('Doskonały dostęp do Darmowych.', -111170), ('Doskonały dostęp do dachu Dachy.', -120190), ('Doskonały dostęp do Darmowych dzwonków.', -118050), ('Doskonały dostęp do Darmowych dzwonków Dzwonki.', -128160)]
[('Doskonały dostęp do Darmowych dzwonków Dzwonki Dla', -13020)]
sent in prog: [('Doskonały dostęp do Darmowych dzwonków Dzwonki Dla', -13020)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Doskonały dostęp do dachu.', -94900), ('Doskonały dostęp do Darmowych.', -111170), ('Doskonały dostęp do dachu Dachy.', -120190), ('Doskonały dostęp do Darmowych dzwonków.', -118050), ('Doskonały dostęp do Darmowych dzwonków Dzwonki.', -128160)]
[('Doskonały dostęp do Darmowych dzwonków Dzwonki Dla', -13020)]
sent in prog: [('Doskonały dostęp do Darmowych dzwonków Dzwonki Dla', -13020)]


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



READY: [('Doskonały dostęp do dachu.', -94900), ('Doskonały dostęp do Darmowych.', -111170), ('Doskonały dostęp do dachu Dachy.', -120190), ('Doskonały dostęp do Darmowych dzwonków.', -118050), ('Doskonały dostęp do Darmowych dzwonków Dzwonki.', -128160)]
[]
sent in prog: []

READY: [('Doskonały dostęp do dachu.', -94900), ('Doskonały dostęp do Darmowych.', -111170), ('Doskonały dostęp do dachu Dachy.', -120190), ('Doskonały dostęp do Darmowych dzwonków.', -118050), ('Doskonały dostęp do Darmowych dzwonków Dzwonki.', -128160)]
[]
sent in prog: []


In [79]:
out4

[('Doskonały dostęp do dachu.', -94900),
 ('Doskonały dostęp do Darmowych.', -111170),
 ('Doskonały dostęp do Darmowych dzwonków.', -118050),
 ('Doskonały dostęp do dachu Dachy.', -120190),
 ('Doskonały dostęp do Darmowych dzwonków Dzwonki.', -128160)]